# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 19 2023 8:45AM,264931,10,38554,"Digital Brands, LLC",Released
1,Jun 19 2023 8:45AM,264931,10,38555,"Digital Brands, LLC",Released
2,Jun 19 2023 8:45AM,264931,10,38556,"Digital Brands, LLC",Released
3,Jun 19 2023 8:45AM,264931,10,38557,"Digital Brands, LLC",Released
4,Jun 19 2023 8:45AM,264931,10,38558,"Digital Brands, LLC",Released
5,Jun 19 2023 8:45AM,264931,10,38559,"Digital Brands, LLC",Released
6,Jun 19 2023 8:45AM,264931,10,38566,"Digital Brands, LLC",Released
7,Jun 19 2023 8:45AM,264931,10,38567,"Digital Brands, LLC",Released
8,Jun 19 2023 8:45AM,264931,10,38568,"Digital Brands, LLC",Released
9,Jun 19 2023 8:45AM,264931,10,38569,"Digital Brands, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,264926,Released,23
8,264927,Released,1
9,264928,Released,1
10,264930,Released,1
11,264931,Released,18


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264926,3.0,23.0
264927,NaN,1.0
264928,NaN,1.0
264930,NaN,1.0
264931,NaN,18.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264921,3.0,1.0
264923,8.0,1.0
264924,0.0,1.0
264925,0.0,1.0
264926,3.0,23.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264921,3,1
264923,8,1
264924,0,1
264925,0,1
264926,3,23


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264921,3,1
1,264923,8,1
2,264924,0,1
3,264925,0,1
4,264926,3,23


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264921,3,1
1,264923,8,1
2,264924,,1
3,264925,,1
4,264926,3,23


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC"
18,Jun 19 2023 8:43AM,264930,16,"SHL Pharma, LLC"
19,Jun 19 2023 8:25AM,264928,19,ThermoFisher Scientific Inc.
20,Jun 19 2023 8:23AM,264927,19,ThermoFisher Scientific Inc.
21,Jun 19 2023 8:15AM,264926,19,"AdvaGen Pharma, Ltd"
47,Jun 19 2023 8:08AM,264925,19,"AdvaGen Pharma, Ltd"
48,Jun 19 2023 7:50AM,264924,19,Opus Life Sciences
49,Jun 19 2023 7:46AM,264923,19,"Methapharm, Inc."
58,Jun 19 2023 7:39AM,264921,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC",,18
1,Jun 19 2023 8:43AM,264930,16,"SHL Pharma, LLC",,1
2,Jun 19 2023 8:25AM,264928,19,ThermoFisher Scientific Inc.,,1
3,Jun 19 2023 8:23AM,264927,19,ThermoFisher Scientific Inc.,,1
4,Jun 19 2023 8:15AM,264926,19,"AdvaGen Pharma, Ltd",3,23
5,Jun 19 2023 8:08AM,264925,19,"AdvaGen Pharma, Ltd",,1
6,Jun 19 2023 7:50AM,264924,19,Opus Life Sciences,,1
7,Jun 19 2023 7:46AM,264923,19,"Methapharm, Inc.",8,1
8,Jun 19 2023 7:39AM,264921,20,"Exact-Rx, Inc.",3,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC",18,
1,Jun 19 2023 8:43AM,264930,16,"SHL Pharma, LLC",1,
2,Jun 19 2023 8:25AM,264928,19,ThermoFisher Scientific Inc.,1,
3,Jun 19 2023 8:23AM,264927,19,ThermoFisher Scientific Inc.,1,
4,Jun 19 2023 8:15AM,264926,19,"AdvaGen Pharma, Ltd",23,3
5,Jun 19 2023 8:08AM,264925,19,"AdvaGen Pharma, Ltd",1,
6,Jun 19 2023 7:50AM,264924,19,Opus Life Sciences,1,
7,Jun 19 2023 7:46AM,264923,19,"Methapharm, Inc.",1,8
8,Jun 19 2023 7:39AM,264921,20,"Exact-Rx, Inc.",1,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC",18,
1,Jun 19 2023 8:43AM,264930,16,"SHL Pharma, LLC",1,
2,Jun 19 2023 8:25AM,264928,19,ThermoFisher Scientific Inc.,1,
3,Jun 19 2023 8:23AM,264927,19,ThermoFisher Scientific Inc.,1,
4,Jun 19 2023 8:15AM,264926,19,"AdvaGen Pharma, Ltd",23,3


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC",18,NaN
1,Jun 19 2023 8:43AM,264930,16,"SHL Pharma, LLC",1,NaN
2,Jun 19 2023 8:25AM,264928,19,ThermoFisher Scientific Inc.,1,NaN
3,Jun 19 2023 8:23AM,264927,19,ThermoFisher Scientific Inc.,1,NaN
4,Jun 19 2023 8:15AM,264926,19,"AdvaGen Pharma, Ltd",23,3.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 19 2023 8:45AM,264931,10,"Digital Brands, LLC",18,0.0
1,Jun 19 2023 8:43AM,264930,16,"SHL Pharma, LLC",1,0.0
2,Jun 19 2023 8:25AM,264928,19,ThermoFisher Scientific Inc.,1,0.0
3,Jun 19 2023 8:23AM,264927,19,ThermoFisher Scientific Inc.,1,0.0
4,Jun 19 2023 8:15AM,264926,19,"AdvaGen Pharma, Ltd",23,3.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,264931,18,0.0
16,264930,1,0.0
19,1589553,28,11.0
20,264921,1,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,264931,18,0.0
1,16,264930,1,0.0
2,19,1589553,28,11.0
3,20,264921,1,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,18,0.0
1,16,1,0.0
2,19,28,11.0
3,20,1,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,18.0
1,16,Released,1.0
2,19,Released,28.0
3,20,Released,1.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20
Status,,,,
Executing,0.0,0.0,11.0,3.0
Released,18.0,1.0,28.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20
0,Executing,0.0,0.0,11.0,3.0
1,Released,18.0,1.0,28.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20
0,Executing,0.0,0.0,11.0,3.0
1,Released,18.0,1.0,28.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()